# 01. LangChain: A Beginner's Guide

This notebook covers the following content: 
1. Loading Model
2. Prompts - `ChatMessage` & `ChatPromptTemplate`
3. Runnables & LangChain Expression Language
4. Chaining 
5. Streaming
6. How to Guide Generation With Context?
7. Debugging - `set_debug`, `set_verbose`, `LangSmith Tracing`, `astream_events`

## 1. Loading Model

### Setup environment variable for LLM API Key

In the root directory of your project, create a file named `.env` to securely store your API key. Add the following line to the `.env` file:
```text
GEMINI_API_KEY = "XXXXXXXXXXXXXXXXX"
```
Make sure to replace "XXXXXXXXXXXXXXXXX" with your actual Gemini API key or your preferred LLM API key (for example, OpenAI or Anthropic).

### Get `.env` variable for LLM API Key

In [1]:
from dotenv import load_dotenv
import os

# Load .env file from the project root
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

if gemini_api_key is None:
    raise ValueError("GEMINI_API_KEY not found. Please set it in the .env file.")

### Initialize the model

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash", 
    temperature = 0, 
    api_key = gemini_api_key
)

In [3]:
response = chat_model.invoke("Tell me a new joke!")
response

AIMessage(content='Okay, here\'s one I just brewed up:\n\nWhy did the AI go to the coffee shop?\n... It heard humans needed a "java update" to start their day!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--d1ab79ff-c12b-4d51-8ce4-8e82bbe7687b-0', usage_metadata={'input_tokens': 7, 'output_tokens': 1406, 'total_tokens': 1413, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 1368}})

The output might seem disorganized with additional metadata, but you can easily extract the text response by using the `.content` attribute

In [4]:
response.content

'Okay, here\'s one I just brewed up:\n\nWhy did the AI go to the coffee shop?\n... It heard humans needed a "java update" to start their day!'

## 2. Prompts

### Using `ChatMessages` from `langchain_core.messages` 

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]

chat_model.invoke(messages).content

'Ciao!'

### Using String `PromptTemplate`

LangChain also supports chat model inputs via strings or `OpenAI` format. The following are equivalent:

In [6]:
# chat_model.invoke("Hello")
# chat_model.invoke([{"role": "user", "content": "Hello"}])
chat_model.invoke([HumanMessage("Hello")]).content

'Hello there! How can I help you today?'

### Using `ChatPromptTemplate` from `langchain_core.prompts`

![image.png](https://www.freecodecamp.org/news/content/images/2024/04/prompt_and_model--1-.png)

In [7]:
from langchain_core.prompts import ChatPromptTemplate 

joke_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class comedian."), 
    ("human", "Tell me a joke about {topic}")
])

In [8]:
joke_prompt.invoke({"topic": "beets"})

ChatPromptValue(messages=[SystemMessage(content='You are a world class comedian.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about beets', additional_kwargs={}, response_metadata={})])

- You construct a prompt template consisting of templates for a `SystemMessage` and a `HumanMessage` using `from_messages`.
- You can think of `SystemMessages` as meta-instructions that are not part of the current conversation, but purely guide input.
- The prompt template contains `{topic}` in curly braces. This denotes a required parameter named `"topic"`.
- You invoke the prompt template with a dict with a key named `"topic"` and a value`"beets"`.
- The result contains the formatted messages.

## 3. Runnables & LangChain Expression Language

**Runnable** in LangChain is a key class that acts as a building block for structuring AI workflows. A Runnable represents a unit of work—like a function or a model call—that can be:
- **Invoked** (run on a single input),
- **Batched** (run on multiple inputs in parallel),
- **Streamed** (produce output as it’s being generated),
- **Transformed** (change its behavior or output),
- **Composed** (combined with other Runnables into workflows).

Runnables provide a standard interface for connecting different steps in your workflow. This means you can easily set up a process where the output from one step is automatically passed as input to the next, creating efficient linear or complex chains of operations.

Runnables are supported by several core methods:
- `invoke / ainvoke`: Run synchronously or asynchronously
- `batch / abatch`: Process many inputs in parallel
- `stream / astream`: Stream output as soon as it’s available

You can build chains using LCEL (LangChain Expression Language) with these components, combining simple tasks into more advanced workflows by methods like RunnableSequence (run steps one after another) or RunnableParallel (run steps at the same time).

What can you do with Runnables?
- `pipe` (or the `|` operator): Connects multiple Runnables in sequence so output flows from one to the next.
- `fromMap`: Runs multiple Runnables at the same time on the same input, useful for parallel processing.
- `passthrough`: Passes the input directly to the output without changes — acts like a placeholder or a no-op.
- `mapInput` and `mapInputStream`: Modify or transform input values or input streams before running the Runnable.
- `fromFunction`: Turns any regular function into a Runnable, allowing easy integration.
`fromRouter`: Routes inputs dynamically to different Runnables based on conditions or logic.

**LangChain Expression Language (LCEL)** is a simple, powerful way to build custom chains of operations using the Runnable protocol. Its core idea is that you can chain any two Runnables together sequentially—the output of one Runnable’s `invoke()` method becomes the input to the next. This chaining can be done using the pipe operator `|` or the equivalent `.pipe()` method, both of which create a `RunnableSequence.`

The resulting `RunnableSequence` is itself a runnable, which means it can be **invoked**, **streamed**, or further **chained** just like any other runnable.

**When to use LCEL?** 
- If you are making a single LLM call, you don't need LCEL; instead call the underlying chat model directly.
- If you have a simple chain (e.g., prompt + llm + parser, simple retrieval set up etc.), LCEL is a reasonable fit, if you're taking advantage of the LCEL benefits.
- If you're building a complex chain (e.g., with branching, cycles, multiple agents, etc.) use LangGraph instead. Remember that you can always use LCEL within individual nodes in LangGraph.

## 4. Chaining

In [9]:
from langchain_core.output_parsers import StrOutputParser

chain = joke_prompt | chat_model | StrOutputParser() 

![image.png](https://www.freecodecamp.org/news/content/images/2024/04/prompt_model_and_output_parser--1-.png)

You still pass `{"topic": "beets"}` as input to the new `chain` because the first `Runnable` in the sequence is still the `PromptTemplate` you declared before.

In [10]:
chain.invoke({"topic": "beets"})

'Alright, alright, settle down folks, you\'re too kind!\n\nSo, I was at a fancy restaurant the other night, and the waiter comes over, all serious, and he\'s describing the beet salad. He says, "It\'s an earthy delight, with a hint of sweetness, and a vibrant color that just *pops*!"\n\nAnd I\'m sitting there, looking at this plate of purple-red stuff, and I just thought to myself...\n\n"Honestly, what *is* it about beets that makes some people absolutely *love* them, and others just can\'t *stand* them?"\n\nAnd then it hit me.\n\n**Beats me!**\n\n*(Pause for groans and applause)*\n\nThank you, I\'ll be here all week! Try the beet salad! Or don\'t! It\'s your life!'

## 5. Streaming

All Runnable objects have two important methods:

- `stream`: a synchronous (sync) method.
- `astream`: an asynchronous (async) variant.

These methods return a generator. A generator lets you get output as soon as it is ready, instead of waiting for everything to finish. This helps your application update quickly when new data becomes available.

**Why Use Streaming?**
- Large language models can take several seconds to reply fully to a query.
- This response time is much slower than the typical 200-300ms it takes for an app to feel instantly responsive to users.

**How to Improve Responsiveness?**
- The main way to keep your app feeling fast is to show progress while you wait for the final result.
- By streaming the model's output token by token (a token is a word or part of a word), you let users see answers as they're being generated.

>**In summary:**
Using `stream` or `astream` allows you to present results immediately and improve the user experience by streaming output progressively as it is created.

### sync `stream` API

In [11]:
chunks = []
for chunk in chat_model.stream("what color is the sky?"): 
    chunks.append(chunk)
    print(chunk.content, end='|', flush=True)

The most common and well-known color of the sky during the day is **|blue**.

However, the sky's color can change dramatically depending on several factors:

*   **Sunrise and Sunset:** It often appears in shades of **red, orange, pink, and purple** due to the way sunlight is scattered when| it travels through more of the Earth's atmosphere at these times.
*   **Overcast or Stormy Weather:** It can look **gray or dark**.
*   **Night:** The sky appears **black or very dark blue**,| with stars visible.
*   **Atmospheric Conditions:** Dust, pollution, or smoke can sometimes give the sky a hazy, yellowish, or brownish tint.

So, while **blue** is the default answer, the sky is actually| a dynamic canvas of many colors!|

>You can also stream output through chains because chains themselves are Runnables. Let's try sync `stream` in chain.

In [12]:
for chunk in chain.stream({"topic": "tiger"}): 
    print(chunk, end='|', flush=True)

Alright, alright, settle down folks, you're too kind! I love this crowd|, you're like a pack of hyenas... but in a good way!

So, I was thinking about tigers the other day, majestic creatures, right? Powerful, graceful... terrible golfers.

No, seriously!

**|Why did the tiger get kicked off the golf course?**

...Because every time he lined up a shot, he'd just pounce on the ball and try to eat it!

*Thank you, I'll be here all| week, try the gazelle!*|

### async `astream` API

If you're working in an async environment, you may consider using the async `astream` API

In [13]:
chunks = [] 
async for chunk in chat_model.astream("what is the color of the sky?"): 
    chunks.append(chunk) 
    print(chunk.content, end='|', flush=True)

The color of the sky is most commonly **blue** during the day.

However, its color can vary significantly depending on the time of day, weather conditions, and atmospheric factors:

|1.  **Blue (Daytime):** This is due to a phenomenon called **Rayleigh scattering**. Sunlight, which appears white, is actually made up of all colors of the rainbow. When sunlight enters Earth's atmosphere, the| shorter wavelengths (like blue and violet) are scattered more efficiently by the tiny nitrogen and oxygen molecules than the longer wavelengths (like red and yellow). Because blue light is scattered in all directions, it makes the sky appear blue to our eyes.

2|.  **Red, Orange, Pink, Yellow (Sunrise/Sunset):** At sunrise and sunset, the sun's light has to travel through much more of the atmosphere to reach our eyes. By the time it gets to us, most of| the blue and violet light has been scattered away, leaving the longer wavelengths (red, orange, yellow) to dominate the sky's color. Dust, pollution

In [14]:
chunks[0]

AIMessageChunk(content='The color of the sky is most commonly **blue** during the day.\n\nHowever, its color can vary significantly depending on the time of day, weather conditions, and atmospheric factors:\n\n', additional_kwargs={}, response_metadata={'safety_ratings': []}, id='run--378aab53-ec21-4c4b-9b6f-60139bcb08d5', usage_metadata={'input_tokens': 9, 'output_tokens': 852, 'total_tokens': 861, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 815}})

## 6. How to Guide Generation with Context?

Large language models (LLMs) are trained on huge amounts of data and contain broad “knowledge” about many topics. However, to get more accurate or specific answers, it’s common to provide the model with extra context or private data relevant to the question. This approach is the core idea behind **Retrieval-Augmented Generation (RAG).**

```
Prompt = Query + Relevant Context  -->  LLM  -->  More Accurate and Useful Response
```

For example, LLMs don’t know the current date because they are trained on data up until a certain cutoff. So, if you want the model to answer based on today’s date, you explicitly supply it as context:

In [15]:
chat_model.invoke("What is the current date and time?").content

"I do not have access to real-time information, including the current date and time. My knowledge cutoff is typically a specific point in the past, and I don't have a built-in clock or the ability to browse the live internet.\n\nTo find the current date and time, please check your device (computer, phone, tablet) or do a quick search online."

Now, let's see what happens when you give the model the current data as context:

In [16]:
from datetime import date 

prompt = ChatPromptTemplate.from_messages([
    ("system", 'You know that the current data is {current_date}.'), 
    ("human", "{question}")
])

chain = prompt | chat_model | StrOutputParser() 

chain.invoke({
    "question": "What is the current date?", 
    "current_date": date.today()
})

'The current date is 2025-07-22.'

Let's try to do something cool

In [17]:
from IPython.display import display, Markdown

response = chain.invoke({
    "question": "I need to prepare for ML interview by the end of this month, give me a short structured plan", 
    "current_date": date.today()
})

display(Markdown(response))

Okay, you have roughly **7-9 days** until the end of July. This is a very tight timeframe, so we need to be highly focused and efficient. This plan assumes you have *some* existing ML knowledge and are looking to refresh and solidify it, not learn from scratch.

**Goal:** Cover the most frequently asked ML interview topics, practice coding, and prepare for behavioral questions.

---

## ML Interview Prep Plan (7-9 Days)

**Core Principle:** Active recall and practice over passive reading. Explain concepts aloud, write code, and simulate interview scenarios.

---

### **Phase 1: ML Fundamentals & Core Algorithms (Days 1-2)**

**Focus:** Solidify theoretical understanding of key concepts and algorithms.

*   **Day 1: ML Basics & Regression/Classification**
    *   **Concepts:**
        *   Supervised vs. Unsupervised Learning
        *   Bias-Variance Tradeoff (Crucial!)
        *   Overfitting & Underfitting, Regularization (L1, L2)
        *   Loss Functions (MSE, Cross-Entropy)
        *   Gradient Descent (Intuition, types: Batch, SGD, Mini-Batch)
    *   **Algorithms:**
        *   **Linear Regression:** Assumptions, how it works, interpretation.
        *   **Logistic Regression:** How it works (sigmoid), probability interpretation, decision boundary.
    *   **Action:**
        *   Review your notes or quick online summaries (e.g., towardsdatascience, GeeksforGeeks).
        *   Explain each concept/algorithm aloud as if teaching someone.
        *   Draw simple diagrams for bias-variance, decision boundaries.

*   **Day 2: Tree-Based Models & Ensemble Methods**
    *   **Algorithms:**
        *   **Decision Trees:** How they work (splitting criteria: Gini, Entropy), pros/cons.
        *   **Random Forests:** Bagging, how it improves DTs, feature importance.
        *   **Gradient Boosting (XGBoost/LightGBM):** Boosting concept, sequential learning, key differences from RF. (Focus on intuition, not deep math).
        *   **SVM (Support Vector Machines):** Intuition of hyperplane, kernel trick (briefly).
    *   **Action:**
        *   Compare and contrast these algorithms: When to use which? What are their strengths/weaknesses?
        *   Understand the difference between bagging and boosting.
        *   Quickly review K-Means (unsupervised clustering) and PCA (dimensionality reduction) intuition.

---

### **Phase 2: Evaluation, Feature Engineering & Practical ML (Days 3-4)**

**Focus:** How to evaluate models, prepare data, and handle practical aspects.

*   **Day 3: Model Evaluation & Hyperparameter Tuning**
    *   **Metrics (Classification):** Accuracy, Precision, Recall, F1-Score, ROC-AUC, Confusion Matrix. (Know when to use which!)
    *   **Metrics (Regression):** MSE, RMSE, MAE, R-squared.
    *   **Techniques:** Cross-Validation (K-Fold), Train-Validation-Test Split.
    *   **Hyperparameter Tuning:** Grid Search, Random Search (briefly).
    *   **Action:**
        *   Work through examples of calculating metrics given a confusion matrix.
        *   Understand the implications of different types of errors (false positives vs. false negatives).
        *   Explain why cross-validation is important.

*   **Day 4: Feature Engineering & Data Preprocessing**
    *   **Concepts:**
        *   Handling Missing Values (Imputation strategies)
        *   Categorical Encoding (One-Hot, Label Encoding)
        *   Feature Scaling (Standardization, Normalization)
        *   Outlier Detection/Handling (briefly)
        *   Feature Selection (basic methods: correlation, importance)
    *   **Action:**
        *   Think about a dataset you've worked with: How would you preprocess it step-by-step?
        *   Practice explaining common feature engineering techniques.
        *   Review `pandas` and `numpy` basics for data manipulation.

---

### **Phase 3: Coding & System Design (Days 5-6)**

**Focus:** Demonstrate your coding proficiency and ability to think about ML in production.

*   **Day 5: Python & ML Libraries Coding**
    *   **Python Basics:** Data structures (lists, dicts, sets, tuples), string manipulation, functions, classes (basic OOP).
    *   **`pandas`:** Data loading, filtering, grouping, merging, pivot tables.
    *   **`numpy`:** Array operations, broadcasting.
    *   **`scikit-learn`:** Model training pipeline (import, instantiate, fit, predict, evaluate).
    *   **Action:**
        *   Solve 2-3 LeetCode Easy/Medium problems (focus on array/string manipulation, hash maps).
        *   Implement a simple ML pipeline using `sklearn` (e.g., train a Logistic Regression on a dummy dataset).
        *   Practice common `pandas` operations (e.g., find top N items, calculate rolling average).

*   **Day 6: ML System Design & Project Deep Dive**
    *   **ML System Design:**
        *   **Concepts:** MLOps lifecycle (data collection, training, deployment, monitoring, retraining).
        *   **Questions:** "How would you design a spam detector?", "How would you build a recommendation system?", "How would you monitor a deployed model for data drift/concept drift?"
        *   **Key Elements:** Data pipelines, model serving, A/B testing, logging, alerting.
    *   **Your Projects:**
        *   **Prepare 1-2 projects** you've worked on in detail.
        *   **STAR Method:** Situation, Task, Action, Result.
        *   **Be ready to discuss:** Why you chose certain models/features, challenges faced, how you evaluated, what you'd do differently.
    *   **Action:**
        *   Brainstorm components for a simple ML system (e.g., a movie recommender).
        *   Practice explaining your projects using the STAR method. Anticipate follow-up questions.

---

### **Phase 4: Mock Interview & Final Review (Day 7-9)**

**Focus:** Consolidate knowledge, practice under pressure, and address weaknesses.

*   **Day 7: Mock Interview & Weakness Targeting**
    *   **Mock Interview:** If possible, do a mock interview with a friend or use a platform like Pramp. This is invaluable for identifying gaps and practicing communication.
    *   **Self-Assessment:** After the mock, or after reviewing the past days, identify your weakest areas.
    *   **Targeted Review:** Spend 2-3 hours diving deeper into those specific weak spots.
    *   **Action:**
        *   Conduct a mock interview.
        *   Prioritize 1-2 topics for deeper review based on your performance.

*   **Day 8-9 (Buffer/Final Polish):**
    *   **Review High-Yield Topics:** Go over your notes for Bias-Variance, Overfitting/Regularization, Key Metrics, and your chosen projects.
    *   **Behavioral Questions:**
        *   "Tell me about yourself." (Craft a concise, compelling story)
        *   "Why ML?" / "Why this company?"
        *   "Strengths/Weaknesses."
        *   "How do you handle conflict/failure?"
    *   **Questions for Interviewer:** Prepare 2-3 thoughtful questions to ask at the end.
    *   **Rest:** Get good sleep! Don't cram the night before.

---

### **Daily Habits (Throughout the 7-9 Days):**

*   **Explain Aloud:** For every concept, try to explain it simply and clearly.
*   **Whiteboard Practice:** Draw diagrams for algorithms, system designs, or data flows.
*   **Quick Code Snippets:** Implement small parts of algorithms or data processing steps.
*   **Review Your Resume/Projects:** Be able to speak confidently about everything listed.
*   **Stay Hydrated & Take Breaks:** Avoid burnout.

---

**What to Skip (for now, given the time constraint):**

*   Deep dives into advanced Deep Learning architectures (Transformers, GANs, etc.) unless the role explicitly requires it.
*   Complex mathematical proofs for algorithms. Focus on intuition.
*   Niche ML algorithms unless they are directly relevant to the job description.

Good luck! You can make significant progress with this focused approach.

>**Note:**
Augmenting generation with additional context is a complex and rich topic. In practical applications, this often involves providing the language model with a large document or relevant sections retrieved from an external data source, such as a financial report. Retrieval-Augmented Generation (RAG) is a powerful technique that enables models to answer questions based on vast amounts of information by first retrieving relevant data and then using it to generate accurate responses.

## 7. Debugging

Language models (LLMs) are inherently non-deterministic, meaning they can produce different outputs on the same input. As your chains grow more complex, understanding the internal data flow becomes critically important for diagnosing issues and improving performance.

There are three main methods for debugging:
- **Verbose Mode**: This adds print statements for "important" events in your chain.
- **Debug Mode**: This add logging statements for ALL events in your chain.
- **LangSmith Tracing**: This logs events to LangSmith to allow for visualization there.

|                       | Verbose Mode | Debug Mode | LangSmith Tracing |
|-----------------------|--------------|------------|-------------------|
| **Free**              | ✅           | ✅         | ✅                |
| **UI**                | ❌           | ❌         | ✅                |
| **Persisted**         | ❌           | ❌         | ✅                |
| **See all events**    | ❌           | ✅         | ✅                |
| **See "important" events** | ✅      | ❌         | ✅                |
| **Runs Locally**      | ✅           | ✅         | ❌                |

>You can check out this [guide](https://python.langchain.com/docs/how_to/debugging/) by LangChain on how to debug your LLM apps


### `set_debug`
LangChain provides a global debugging utility called `set_debug()` that enables detailed logging of all chain internals. When enabled, it logs every input and output passing through components like chains, models, agents, tools, and retrievers, helping you trace exactly what data is processed step-by-step.

Install `LangChain` if you haven't already.
```
pip install langchain
```

In [18]:
from langchain.globals import set_debug
from datetime import date

set_debug(True)

prompt = ChatPromptTemplate.from_messages([
    ("system", 'You know that the current date is "{current_date}".'),
    ("human", "{question}")
])

chain = prompt | chat_model | StrOutputParser()

chain.invoke({
    "question": "What is the current date?",
    "current_date": date.today()
})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System: You know that the current date is \"2025-07-22\".\nHuman: What is the current date?"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The current date is July 22, 2025.",
        "generation_info": {
          "finish_reason": "STOP",
          "model_name": "gemini-2.5-flash",
          "safety_ratings": []
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
  

'The current date is July 22, 2025.'

### `set_verbose`
Setting the verbose flag will print out inputs and outputs in a slightly more readable format and will skip logging certain raw outputs (like the token usage stats for an LLM call) so that you can focus on application logic.

In [19]:
from langchain.globals import set_verbose
from datetime import date

set_debug(False)
set_verbose(True) 

prompt = ChatPromptTemplate.from_messages([
    ("system", 'You know that the current date is "{current_date}".'),
    ("human", "{question}")
])

chain = prompt | chat_model | StrOutputParser()

chain.invoke({
    "question": "What is the current date?",
    "current_date": date.today()
})


'The current date is July 22, 2025.'

>**Note**: Debug verbosity is now deprecated in many core Runnables/chains—so it doesn’t always trigger output. It only on objects that support it, like AgentExecutor or when you pass verbose=True to a Runnable or agent—not on simple pipeline chains like prompt | model | parser.

In [20]:
set_verbose(False)

### LangSmith Tracing

You can enhance your LangChain applications by adding tracing with an external service like **LangSmith**—a powerful platform designed for debugging, evaluating, and monitoring LLM apps.

**What is LangSmith?** \
LangSmith provides end-to-end observability for your language model workflows:
- Tracks all inputs and outputs at every step
- Helps you debug and test chains and agents effectively
- Enables prompt engineering with version control and collaboration
- Offers dashboards to monitor metrics like request rates, error rates, and costs
- Supports automated evaluations and human-in-the-loop feedback

**How to Add LangSmith Tracing?**
1. [Sign up](https://smith.langchain.com/) and get your LangSmith API key from the LangSmith dashboard.
2. Set environment variables to enable tracing and provide your API key:

After you sign up at the link above, make sure to set your environment variables to start logging traces:
```
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

chain.invoke({
  "question": "What is the current date?",
  "current_date": date.today()
}) 
```

### `astream_events()`

You can use the `astream_events()` method in LangChain to asynchronously receive detailed event data during the execution of your chains or models. This is especially useful when you want to integrate intermediate outputs or internal states directly into your application’s logic.

Unlike simple streaming methods that yield only output tokens, `astream_events()` returns an `async` iterator over a variety of event objects, such as model start, intermediate token generation, and tool calls. This lets you selectively process or respond to specific event types as they occur in real-time.

In [21]:
# Turn off debug mode for clarity
set_debug(False)

stream = chain.astream_events({
    "question": "What is the current date?",
    "current_date": date.today()
})

async for event in stream:
    print(event)
    print("-----")

{'event': 'on_chain_start', 'data': {'input': {'question': 'What is the current date?', 'current_date': datetime.date(2025, 7, 22)}}, 'name': 'RunnableSequence', 'tags': [], 'run_id': 'df18db9b-cd65-4d59-a28e-94e5e5051cee', 'metadata': {}, 'parent_ids': []}
-----
{'event': 'on_prompt_start', 'data': {'input': {'question': 'What is the current date?', 'current_date': datetime.date(2025, 7, 22)}}, 'name': 'ChatPromptTemplate', 'tags': ['seq:step:1'], 'run_id': 'a21a53cb-187f-4dc7-9583-d3035f983c98', 'metadata': {}, 'parent_ids': ['df18db9b-cd65-4d59-a28e-94e5e5051cee']}
-----
{'event': 'on_prompt_end', 'data': {'output': ChatPromptValue(messages=[SystemMessage(content='You know that the current date is "2025-07-22".', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the current date?', additional_kwargs={}, response_metadata={})]), 'input': {'question': 'What is the current date?', 'current_date': datetime.date(2025, 7, 22)}}, 'run_id': 'a21a53cb-187f-4dc7-9583-

# Thank you!
Congratulations on completing 

### References 
- freecodecamp - [How to Use LangChain to Build With LLMs – A Beginner's Guide](https://www.freecodecamp.org/news/beginners-guide-to-langchain/)
- LangChain - [How-to guides](https://python.langchain.com/docs/how_to/)